# ML Model to Detect Fraudelent Transactions

## Connecting to kaggle

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vijaybalaji123","key":"717d5307d8e54e0382c83fc4d95a3945"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                          title                                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
haseebindata/student-performance-predictions                 Student Performance Predictions                     9KB  2024-08-17 06:57:57           8290        179  0.9411765        
lainguyn123/student-performance-factors                      Student Performance Factors                        94KB  2024-09-02 10:53:57           2017         48  1.0              
lainguyn123/employee-survey                                  Employee Survey                                    61KB  2024-09-03 09:46:24            918         23  1.0              
abdullah0a/human-age-prediction-synthetic-dataset            Human Age Prediction Syn

## Downloadin the required dataset from Kaggle

In [ ]:
! kaggle datasets download -d rupakroy/online-payments-fraud-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/rupakroy/online-payments-fraud-detection-dataset
License(s): CC-BY-NC-SA-4.0
 96% 171M/178M [00:02<00:00, 70.6MB/s]
100% 178M/178M [00:02<00:00, 78.4MB/s]


In [ ]:
! unzip online-payments-fraud-detection-dataset.zip

Archive:  online-payments-fraud-detection-dataset.zip
  inflating: PS_20174392719_1491204439457_log.csv  


## Importing pandas and doing some data preprocessing

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")

In [ ]:
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [ ]:
df["nameOrig_type"]=df["nameOrig"].str[:1]

In [ ]:
df["nameOrig_type"].unique()

array(['C'], dtype=object)

In [ ]:
# since we have only one type of unique values in the type of the original account it wont be a useful feature in our model
df=df.drop(columns=["nameOrig","nameOrig_type"])
df

,step,type,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [ ]:
df["dest_type"]=df["nameDest"].str[:1]

In [ ]:
df["dest_type"].unique()

array(['M', 'C'], dtype=object)

In [ ]:
df=df.drop(columns=["nameDest"])

In [ ]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(df['type'])
# Drop column B as it is now encoded
df = df.drop('type',axis = 1)
# Join the encoded df
df = df.join(one_hot)



one_hot=pd.get_dummies(df["dest_type"])
df=df.drop(columns=["dest_type"])
df=df.join(one_hot)

In [ ]:
df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,C,M
0,1,9839.64,170136.00,160296.36,0.00,0.00,0,0,False,False,False,True,False,False,True
1,1,1864.28,21249.00,19384.72,0.00,0.00,0,0,False,False,False,True,False,False,True
2,1,181.00,181.00,0.00,0.00,0.00,1,0,False,False,False,False,True,True,False
3,1,181.00,181.00,0.00,21182.00,0.00,1,0,False,True,False,False,False,True,False
4,1,11668.14,41554.00,29885.86,0.00,0.00,0,0,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,339682.13,339682.13,0.00,0.00,339682.13,1,0,False,True,False,False,False,True,False
6362616,743,6311409.28,6311409.28,0.00,0.00,0.00,1,0,False,False,False,False,True,True,False
6362617,743,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0,False,True,False,False,False,True,False
6362618,743,850002.52,850002.52,0.00,0.00,0.00,1,0,False,False,False,False,True,True,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 15 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   amount          float64
 2   oldbalanceOrg   float64
 3   newbalanceOrig  float64
 4   oldbalanceDest  float64
 5   newbalanceDest  float64
 6   isFraud         int64  
 7   isFlaggedFraud  int64  
 8   CASH_IN         bool   
 9   CASH_OUT        bool   
 10  DEBIT           bool   
 11  PAYMENT         bool   
 12  TRANSFER        bool   
 13  C               bool   
 14  M               bool   
dtypes: bool(7), float64(5), int64(3)
memory usage: 430.8 MB


In [ ]:
df.isnull().sum()   # checking for any null values

,0
step,0
amount,0
oldbalanceOrg,0
newbalanceOrig,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0
isFlaggedFraud,0
CASH_IN,0
CASH_OUT,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
columns=["step","amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest"]
scaler=MinMaxScaler()
df[columns]=scaler.fit_transform(df[columns])    #normalising all the columns

In [ ]:
df

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,C,M
0,0.0,0.000106,0.002855,0.003233,0.000000,0.000000,0,0,False,False,False,True,False,False,True
1,0.0,0.000020,0.000357,0.000391,0.000000,0.000000,0,0,False,False,False,True,False,False,True
2,0.0,0.000002,0.000003,0.000000,0.000000,0.000000,1,0,False,False,False,False,True,True,False
3,0.0,0.000002,0.000003,0.000000,0.000059,0.000000,1,0,False,True,False,False,False,True,False
4,0.0,0.000126,0.000697,0.000603,0.000000,0.000000,0,0,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362615,1.0,0.003674,0.005701,0.000000,0.000000,0.000954,1,0,False,True,False,False,False,True,False
6362616,1.0,0.068272,0.105923,0.000000,0.000000,0.000000,1,0,False,False,False,False,True,True,False
6362617,1.0,0.068272,0.105923,0.000000,0.000192,0.017912,1,0,False,True,False,False,False,True,False
6362618,1.0,0.009195,0.014265,0.000000,0.000000,0.000000,1,0,False,False,False,False,True,True,False


In [ ]:
df["isFlaggedFraud"].unique() # checking is the column "isFlaggedFraud" always 0

array([0, 1])

In [ ]:
df["isFlaggedFraud"].sum()

16

In [ ]:
df["isFlaggedFraud"]=0 # making the column "isFlaggedFraud" always zero as it a column that we should update.

In [ ]:
df_x=df.drop(columns=["isFraud"])   #input dataset
df_y=df["isFraud"]                  #output dataset

## Splitting the dataset for training and testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_x,df_y,test_size=0.2,random_state=42)

## Importing the required libraries for creating Random forest model

In [ ]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)

##checking how well the model fares

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy=", accuracy)

The dataset in which the above model was trained had class imbalance as there is very less number of fradulent transaction compared to the number of genuine transactions.
So to deal with the we are going to undersample a new model with equal number of genuine and fraudulent transactions.

In [ ]:
df1=df[df["isFraud"]==1]
df1

In [ ]:
df2=df[df["isFraud"]==0].sample(len(df1))
df2

In [ ]:
df3=pd.concat([df2,df1])
df3

In [ ]:
df3_x=df3.drop(columns=["isFraud"])
df3_y=df3["isFraud"]
X_train,X_test,y_train,y_test=train_test_split(df3_x,df3_y,test_size=0.3)

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy=",accuracy)

##Tuning the hyper parameters

In [ ]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf,
                                 param_distributions = param_dist,
                                 n_iter=5,
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

In [ ]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

#XGBoost Model

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
! pip install --user xgboost
# ! conda install -c conda-forge py-xgboost-gpu

In [ ]:
df=pd.read_csv("PS_20174392719_1491204439457_log.csv")

In [ ]:
df

In [ ]:
# X=df.drop(columns=["isFraud"])
def normalize(column):
  min_val=column.min()
  max_val=column.max()
  return (column-min_val)/(max_val-min_val)
for column in ["step","amount","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest"]:
  df[column]=normalize(df[column])
df1=df[df["isFraud"]==1]
df2=df[df["isFraud"]==0].sample(len(df1))
df3=pd.concat([df1,df2])
X=df3.drop(columns=["isFraud"])
y=df3["isFraud"]
# y=df["isFraud"]

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')


In [ ]:
X.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
import numpy as np

# mse = np.mean((actual - predicted) ** 2)
# rmse = np.sqrt(mse)

In [ ]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

In [ ]:
n = 100
model = xgb.train(params=params,dtrain=dtrain_reg,num_boost_round=n)


In [ ]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)


In [ ]:
rmse = mean_squared_error(y_test, preds, squared=False)

In [ ]:
print(f"RMSE of the base model: {rmse:.3f}")

In [ ]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

In [ ]:
n=100
model=xgb.train(params=params,dtrain=dtrain_reg,num_boost_round=n,evals=evals)

In [ ]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]


model = xgb.train(params=params,dtrain=dtrain_reg,num_boost_round=n,evals=evals,verbose_eval=10,early_stopping_rounds=5) # Every ten rounds)
